In [1]:
from pathlib import Path
import pandas as pd
from inference_gpt import generate_gpt_report
import os
from dotenv import load_dotenv
import pickle

load_dotenv()
processed_data_path = Path(os.getcwd()) / "output" / "cleaned"
temp_file_path = Path(os.getcwd()) / "temp-files"
# event = "event2_anchor_new"
event = "event2multianchor_stakeholders"
anchor_file_path_dir = Path("/mnt/datasets/dop-position-mining/wiki-anchor/")
# check if env variable LSIR_OPENAI_PROJ_KEY is set
assert "LSIR_OPENAI_PROJ_KEY" in os.environ, "Please set the environment variable LSIR_OPENAI_PROJ_KEY"

In [2]:
# df = pd.read_json(processed_data_path / f"{event}.json")

In [3]:
with open(temp_file_path / f"augmented_stakeholders_{event}.pkl", "rb") as f:
    augmented_stakeholders = pickle.load(f)

with open(temp_file_path / f"stakeholders_{event}.pkl", "rb") as f:
    stakeholders_early = pickle.load(f)

with open(temp_file_path / f"results_{event}.pkl", "rb") as f:
    wiki_results = pickle.load(f)

with open(temp_file_path / f"stakeholder_to_wiki_info_{event}.pkl", "rb") as f:
    stakeholder_to_wiki_info = pickle.load(f)

with open(temp_file_path / f"missing_stakeholder_matches_{event}.pkl", "rb") as f:
    missing_stakeholder_matches = pickle.load(f)

with open(temp_file_path / f"missing_stakeholder_replacements_{event}.pkl", "rb") as f:
    missing_stakeholder_replacements = pickle.load(f)

with open(temp_file_path / f"all_wiki_info_{event}.pkl", "rb") as f:
    all_wiki_info = pickle.load(f)

with open(temp_file_path / f"wiki_info_index_to_stakeholder_{event}.pkl", "rb") as f:
    wiki_info_index_to_stakeholder = pickle.load(f)

with open(temp_file_path / f"replacement_{event}.pkl", "rb") as f:
    stakeholder_replacement = pickle.load(f)

In [6]:
from collections import defaultdict
wiki_results_grouped = defaultdict(list)

for key,value in wiki_results.items():
    wiki_results_grouped[value[0]].append(key)

In [7]:
wiki_results_grouped

defaultdict(list,
            {'donald trump jr.': ['donald trump jr.'],
             'vasyl durdynets': ['vasyl dudynets'],
             'james r. nix': ['james nixey'],
             'marco asensio': ['marco asensio'],
             'european union': ['european union', 'union européenne'],
             'kathleen mcinerney': ['kathleen mcinnis'],
             'united kingdom ministry of defence': ['ministry of defence of the united kingdom',
              'united kingdom ministry of defence'],
             'round valley indian tribes': ['round valley indian tribes'],
             'peter rindisbacher': ['stephan rindlisbacher'],
             'jeffrey kofman': ['michael kofman', 'alexander kofman'],
             'oleksandr klymenko': ['ihor klymenko',
              'igor klymenko',
              'andrii klymenko'],
             'ambassador of russia to ukraine': ['ukraine osce ambassador',
              "ukraine's un envoy"],
             'michael kaufman': ['michael kaufman'],
          

In [8]:
missing_stakeholder_matches["wolodimir selenski"]

('wolodymyr selenskyj', 0.8902431130409241, 'vladimir zelensky')

In [9]:
print(all_wiki_info[:5], wiki_info_index_to_stakeholder[:5])

['donald trump, jr.', 'donald trump jr.', 'donald trump jr.', 'donald trump, jr.', 'donald trump jr.'] ['donald trump jr.', 'donald trump jr.', 'donald trump jr.', 'donald trump jr.', 'donald trump jr.']


In [10]:
for wiki_info in all_wiki_info:
    if ('president' in wiki_info.lower()) and ('ukraine' in wiki_info.lower()):
        print(wiki_info)

volodymyr zelenskyy president of ukraine since 2019
volodymyr zelenskyy president of ukraine since 2019
volodymyr zelenskyy president of ukraine since 2019
volodymyr zelenskyy president of ukraine since 2019
volodymyr zelenskyy president of ukraine since 2019
volodymyr zelenskyy president of ukraine since 2019
volodymyr zelenskyy president of ukraine since 2019
volodymyr zelenskyy president of ukraine since 2019
volodymyr zelenskyy president of ukraine since 2019
volodymyr zelenskyy president of ukraine since 2019
president of ukraine
president of ukraine head of state of ukraine
volodymyr zelenskyy president of ukraine since 2019
volodymyr zelenskyy president of ukraine since 2019
volodymyr zelenskyy president of ukraine since 2019
representatives of the president of ukraine
representatives of the president of ukraine 
national security and defense council of ukraine advisory state body to the president of ukraine
volodymyr zelenskyy president of ukraine since 2019
petro poroshenko 5t

In [11]:
for wiki_info in all_wiki_info:
    if ('zelen' in wiki_info.lower()) or ('selen' in wiki_info.lower()):
        print(wiki_info)

volodymyr zelenskyy
wolodymyr selenskyj
volodimir zelens'kij‏
volodymyr zelensky
volodímir zelenski
volodimir zelenski
volodymyr zelenskyy president of ukraine since 2019
vladimir zelenskiy
volodymyr zelensky
volodymyr zelenskyy
volodimir zelenski
volodimir oleksandrovič zelens'kij
vladimir zelensky
volodimir oleksandroviç zelenski
volodımır zelenskıy
vladimir zelenski
volodymyr zelenskiy
vladimir zelenskiy
volodymyr zelensky
vladimir aleksandrovitch zelenski
volodymyr oleksandrovytch zelensky
vladimir zelenski
volodymyr zelensky
volodymyr zelenskiy
volodymyr oleksandrovych zelenskyy
president volodymyr zelenskyy
president volodymyr oleksandrovych zelenskyy
president zelenskyy
volodymyr zelenskyj
volodymyr zelenskyi
wolodymyr oleksandrowytsch selenskyj
volodymyr zelenskyy
wolodymyr selenskyj
volodimir zelens'kij‏
volodymyr zelensky
volodímir zelenski
volodimir zelenski
volodymyr zelenskyy president of ukraine since 2019
vladimir zelenskiy
volodymyr zelensky
volodymyr zelenskyy
volodimi

In [12]:
missing_stakeholders = list(set(stakeholders_early) - set(wiki_results.keys()))
print(missing_stakeholders[:5])

['atesh guerrillas', 'ukrainian geographical society', 'two majors telegram channel', 'gerardo berthin', 'bryansk governor']


In [13]:
hit_count = sum([1 for stakeholder in stakeholders_early if stakeholder in wiki_results])
print(f"Hit count: {hit_count} | Hit percentage before wiki: {hit_count / len(stakeholders_early) * 100:.2f}%")

Hit count: 11953 | Hit percentage before wiki: 83.37%


In [14]:
stakeholders_with_zelen = []

for stakeholder in stakeholders_early:
    if ('zelen' in stakeholder.lower()) or ('selen' in stakeholder.lower()):
        stakeholders_with_zelen.append(stakeholder)

print(set(stakeholders_with_zelen))

{'olena zelensky', 'vlodomyr zelensky', 'president volodymyr zelensky', 'volodymyr zelenski', 'volodomyr zelenskiy', 'wolodimir zelenski', 'wolodymyr selenskyj', 'volodymyr zelenskyy', 'wolodimir selenski', 'olena zelenska', 'volodymyr zelenskyi', 'volodimir zelenski', 'volodimir zelensky', 'volodymyr zelenskiyy', 'vladimir zelensky', 'vladimir zelenskyy', 'former zelensky administration official', 'volodymyr zelenskyj', 'volodymyr zelenskiy', 'wolodimir selenskij', 'vladimir zelenskiy', 'zelenskyy', 'vladimir aleksandrovich zelensky', 'ukrainian president volodymyr zelensky', 'wolodymyr zelenskyj', 'volodymir zelensky', 'volodymyr zelensky', 'wolodymyr zelensky'}


In [15]:
for stakeholder in set(stakeholders_with_zelen):
    if stakeholder in wiki_results:
        print(stakeholder, wiki_results[stakeholder])
    else:
        print(f"{stakeholder} not found")

olena zelensky not found
vlodomyr zelensky ('volodymyr zelenskiy', 59210523, 3874799)
president volodymyr zelensky ('volodymyr zelensky', 59210523, 3874799)
volodymyr zelenski ('volodymyr zelenskiy', 59210523, 3874799)
volodomyr zelenskiy ('volodymyr zelenskiy', 59210523, 3874799)
wolodimir zelenski ('volodymyr zelenskiy', 59210523, 3874799)
wolodymyr selenskyj not found
volodymyr zelenskyy ('volodymyr zelenskiy', 59210523, 3874799)
wolodimir selenski not found
olena zelenska not found
volodymyr zelenskyi ('volodymyr zelenskyi', 59210523, 3874799)
volodimir zelenski ('volodymyr zelenskiy', 59210523, 3874799)
volodimir zelensky ('vladimir zelensky', 59210523, 3874799)
volodymyr zelenskiyy ('volodymyr zelenskiy', 59210523, 3874799)
vladimir zelensky ('vladimir zelensky', 59210523, 3874799)
vladimir zelenskyy ('vladimir zelensky', 59210523, 3874799)
former zelensky administration official not found
volodymyr zelenskyj ('volodymyr zelenskyi', 59210523, 3874799)
volodymyr zelenskiy ('volody

In [11]:
df_max_views = pd.read_parquet(anchor_file_path_dir / "df_max_views.parquet")

In [18]:
import torch
from post_processing_new import semantic_search
from transformers import AutoModel

model = AutoModel.from_pretrained(
        "arkohut/jina-embeddings-v3", trust_remote_code=True
).to("cuda")

unique_stakeholders = list(set(stakeholders_early))

/home/yazici/.cache/pypoetry/virtualenvs/new-prompts-cm8HmoVy-py3.9/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native

In [17]:
# Initial wiki matches
unique_stakeholder_embeddings = model.encode(
    unique_stakeholders,
    show_progress_bar=True,
    convert_to_tensor=True,
    device="cuda",
    batch_size=64*6,
)
all_wiki_info_unique = list(set(all_wiki_info))
all_wiki_info_embeddings = model.encode(
    all_wiki_info_unique,
    show_progress_bar=True,
    convert_to_tensor=True,
    device="cuda",
    batch_size=64*6,
)

flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn i

Encoding:   0%|          | 0/10 [00:00<?, ?it/s]

Encoding:   0%|          | 0/82 [00:00<?, ?it/s]

In [18]:
search_results = semantic_search(
    unique_stakeholder_embeddings,
    all_wiki_info_embeddings,
    top_k=5,
    device="cuda",
)

Query chunks:   0%|          | 0/39 [00:00<?, ?it/s]

Query chunks: 100%|██████████| 39/39 [00:00<00:00, 360.06it/s]


In [19]:
for idx, result in enumerate(search_results):
    stakeholder = unique_stakeholders[idx]
    result_new = [
        (all_wiki_info_unique[x["corpus_id"]], x["score"]) for x in result
    ]
    if stakeholder == 'wolodimir selenski':
        print(stakeholder, result_new)
        print("\n")

wolodimir selenski [('Wolodymyr Selenskyj', 0.837627649307251), ('Iwan Seliminski', 0.8299015760421753), ('Wolodymyr Oleksandrowytsch Selenskyj', 0.8119789958000183), ('Wladimir Selesnjow', 0.8106221556663513), ('Iwan Georgiew Seliminski', 0.8012471199035645)]




In [13]:
df_embeddings = torch.stack(
    [torch.tensor(emb) for emb in df_max_views["embedding"].values]
)

In [26]:
search_results2 = semantic_search(
    unique_stakeholder_embeddings,  # query embeddings
    df_embeddings,  # database embeddings (can remain on CPU)
    top_k=5,  # Retrieve the top match only, adjust as needed,
    device="cuda",
)

for idx, result in enumerate(search_results2):
    stakeholder = unique_stakeholders[idx]
    result_new = [
        (df_max_views.iloc[x["corpus_id"]]["normalized_anchor_text"], x["score"]) for x in result
    ]
    if stakeholder == 'wolodimir selenski':
        print(stakeholder, result_new)
        print("\n")

wolodimir selenski [('ivan seliminski', 0.8781840801239014), ('vladimir belinski', 0.8659109473228455), ('emil selenka', 0.8595795631408691), ('selenginsky', 0.8553673028945923), ('simon lenski', 0.8527318239212036)]




In [22]:
from openai import OpenAI
import torch
from tqdm.notebook import tqdm

def get_embedding_openai(text_array, model="text-embedding-3-small", client=None, batch_size=800):
    if client is None:
        client = OpenAI(api_key=os.environ["LSIR_OPENAI_PROJ_KEY"])
    embeds = []
    for i in tqdm(range(0, len(text_array), batch_size), desc="Embedding"):
        batch = text_array[i:i+batch_size]
        # replace empty strings with '<PAD>'
        batch = [text if text != "" else "<PAD>" for text in batch]
        response = client.embeddings.create(input=batch, model=model)
        response_array = response.data
        for element in response_array:
            embeds.append(element.embedding)
    return torch.tensor(embeds) 

In [44]:
test_str1 = "wolodymyr selenskyj"
test_str2 = 'wolodimir selenski'
test_str3 = "wolodimir selenskij"
test_str4 = "vladimir seleznev"
embeds = model.encode(
    [test_str1, test_str2, test_str3, test_str4],
    convert_to_tensor=True,
    device="cuda",
    task="text-matching",
    truncate_dim=1024,
)

embed_str1 = embeds[0]
embed_str2 = embeds[1]
embed_str3 = embeds[2]
embed_str4 = embeds[3]


In [28]:
openai_embed = get_embedding_openai([test_str1, test_str2, test_str3, test_str4], model="text-embedding-3-large")

openai_embed_str1 = openai_embed[0]
openai_embed_str2 = openai_embed[1]
openai_embed_str3 = openai_embed[2]
openai_embed_str4 = openai_embed[3]

from sentence_transformers import SentenceTransformer
all_mpnet_model = SentenceTransformer("all-mpnet-base-v2", device="cuda")

all_mpnet_embed = all_mpnet_model.encode([test_str1, test_str2, test_str3, test_str4])

all_mpnet_embed_str1 = all_mpnet_embed[0]
all_mpnet_embed_str2 = all_mpnet_embed[1]
all_mpnet_embed_str3 = all_mpnet_embed[2]
all_mpnet_embed_str4 = all_mpnet_embed[3]

Embedding:   0%|          | 0/1 [00:00<?, ?it/s]

In [45]:
from sentence_transformers import util

print("JINA 1-2:", util.cos_sim(embed_str1, embed_str2))
print("JINA 3-4:",util.cos_sim(embed_str3, embed_str4))
print("JINA 1-3:",util.cos_sim(embed_str1, embed_str3))

JINA 1-2: tensor([[0.8642]], device='cuda:0')
JINA 3-4: tensor([[0.8453]], device='cuda:0')
JINA 1-3: tensor([[0.9229]], device='cuda:0')


In [33]:
print("OPENAI 1-2:",util.cos_sim(openai_embed_str1, openai_embed_str2))
print("OPENAI 3-4:",util.cos_sim(openai_embed_str3, openai_embed_str4))
print("OPENAI 1-3:",util.cos_sim(openai_embed_str1, openai_embed_str3))

OPENAI 1-2: tensor([[0.8708]])
OPENAI 3-4: tensor([[0.3921]])
OPENAI 1-3: tensor([[0.8953]])


In [34]:
print("MPNET 1-2:",util.cos_sim(all_mpnet_embed_str1, all_mpnet_embed_str2))
print("MPNET 3-4:",util.cos_sim(all_mpnet_embed_str3, all_mpnet_embed_str4))
print("MPNET 1-3:",util.cos_sim(all_mpnet_embed_str1, all_mpnet_embed_str3))

MPNET 1-2: tensor([[0.8550]])
MPNET 3-4: tensor([[0.5061]])
MPNET 1-3: tensor([[0.8940]])


In [35]:
# embed averages
embed_avg = (embeds + openai_embed + all_mpnet_embed) / 3

RuntimeError: The size of tensor a (1024) must match the size of tensor b (3072) at non-singleton dimension 1

In [6]:
from collections import defaultdict
stakeholder_replacement_grouped = defaultdict(list)
for k,v in stakeholder_replacement.items():
    stakeholder_replacement_grouped[v].append(k)

In [10]:
key_we_are_looking_for = stakeholder_replacement['Volodymyr Zelenskyy']
print(stakeholder_replacement_grouped[key_we_are_looking_for])

['Volodymyr Zelenskyy']


In [28]:
key_we_are_looking_for = stakeholder_replacement['Vladimir Putin']
print(stakeholder_replacement_grouped[key_we_are_looking_for])

['Vladimir Putin']


In [9]:
for k,v in stakeholder_replacement_grouped.items():
    if len(v) >= 2:
        print(k,v)

Ministry of Foreign Affairs of Ukraine ['Ministry of Foreign Affairs of Ukraine', 'Ministry of Internal Affairs of Ukraine', 'Ministry of Defence of Ukraine']
Central Election Commission of Russia ['Central Election Commission of Russia', 'Central Election Commission of Ukraine']
dmytro kuleba ['dmytro kuleba', 'dmitro kuleba', 'dimitro kuleba', 'dmitri kuleba', 'dmytro kouleba', 'dymtro kuleba', 'dmitry ivanovich kuleba', 'dmitro koeleba', 'oleksiy kuleba']
hanna malyar ['hanna malyar', 'hanna maljar', 'hanna malijar', 'hanna maliar', 'hannah maliar', 'anna malyar']
mykhailo podolyak ['mykhailo podolyak', 'mykhailo podoliak', 'mikhail podolyak', 'mychailo podolyak', 'miikhail podoliak', 'mikhail podoliak', 'mychailo podoljak', 'mijailo podoliak', 'mykahilo podolyak']
vladimir saldo ['vladimir saldo', 'vadimir saldo', 'volodymyr saldo']
General Staff of the Ukrainian Armed Forces ['General Staff of the Ukrainian Armed Forces', 'General of Army of Ukraine']
Embassy of Russia, Washington

In [8]:
missing_stakeholder_replacements_grouped = defaultdict(list)
for k,v in missing_stakeholder_replacements.items():
    missing_stakeholder_replacements_grouped[v].append(k)

In [9]:
missing_stakeholder_replacements["wolodimir selenski"]

'vlodomyr zelensky'

<hr/>

In [18]:
# replace empty strings with '<PAD>' in all_wiki_info
all_wiki_info = [wiki_info if wiki_info != "" else "<PAD>" for wiki_info in all_wiki_info]
len(all_wiki_info)

41473

In [19]:
from data_parallel import get_embeddings_model_parallel
from post_processing_new import semantic_search
import torch
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer

# Load the tokenizer and model
model_name = "arkohut/jina-embeddings-v3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# map the model to 8GB to device0, 8GB to device1 and 8GB to device2
# model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to("cuda:0")
model = SentenceTransformer("all-mpnet-base-v2", device="cuda:1")

/home/yazici/.cache/pypoetry/virtualenvs/new-prompts-cm8HmoVy-py3.9/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [12]:
# all_wiki_embeds = get_embeddings(all_wiki_info, model, tokenizer=tokenizer, batch_size=64)

all_wiki_embeds = model.encode(
    all_wiki_info,
    show_progress_bar=True,
    convert_to_tensor=True,
    batch_size=64 * 32,
    device="cuda:1",
)

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

In [13]:
all_wiki_embeds.shape

torch.Size([44351, 768])

In [14]:
missing_stakeholder_embeddings = model.encode(
    missing_stakeholders,
    convert_to_tensor=True,
    show_progress_bar=True,
    device="cuda",
    batch_size=64 * 64,
    task="text-similarity"
)
# missing_stakeholder_embeddings = get_embeddings_model_parallel(missing_stakeholders, model, tokenizer=tokenizer, batch_size=128)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
missing_stakeholder_embeddings.shape

torch.Size([1246, 768])

In [18]:
from torch import topk
cos_scores = missing_stakeholder_embeddings.to("cuda:2") @ all_wiki_embeds.to("cuda:2").T
print("Cos scores shape:", cos_scores.shape)

topk_values, topk_indices = topk(cos_scores, 1, dim=1)
search_results = [{"corpus_id": index, "score": topk_values[i].item()} for i, (index,) in enumerate(topk_indices)]

Cos scores shape: torch.Size([1246, 44351])


In [19]:
# search_results = semantic_search(
#     missing_stakeholder_embeddings,  # query embeddings
#     all_wiki_embeds,  # database embeddings (can remain on CPU)
#     top_k=1,  # Retrieve the top match only, adjust as needed,
#     device='cuda',
# )

In [20]:
missing_stakeholder_matches = {}
missing_stakeholder_replacements = {}
for idx, result in enumerate(search_results):
    best_match = result
    similarity_score = best_match['score']
    best_match_idx = best_match['corpus_id']
    missing_stakeholder_matches[missing_stakeholders[idx]] = (all_wiki_info[best_match_idx], similarity_score, wiki_info_index_to_stakeholder[best_match_idx])
    print_string = "Mapping: " if similarity_score >= (1 - float(0.15)) else "Closest Match: "
    print(f"{print_string} {missing_stakeholders[idx]} -> {all_wiki_info[best_match_idx]} | Similarity: {similarity_score:.4f} | Wiki stakeholder: {wiki_info_index_to_stakeholder[best_match_idx]}")
    if similarity_score >= (1 - float(0.17)):
        stakeholder = missing_stakeholders[idx]
        best_match_stakeholder = wiki_info_index_to_stakeholder[best_match_idx]
        missing_stakeholder_replacements[stakeholder] = best_match_stakeholder

Closest Match:  harald wilimsky -> Sergei Wladilenowitsch Kirijenko | Similarity: 0.6980 | Wiki stakeholder: sergey kiriyenko
Closest Match:  azam nazeer tarar -> Imran Ahmed Khan Niazi | Similarity: 0.6292 | Wiki stakeholder: imran khan
Closest Match:  kiselev ruslan -> Ruslan Koshulynskyy | Similarity: 0.8259 | Wiki stakeholder: ruslan koshulynsky
Closest Match:  naeem haider panjhuta -> Najeebullah Khan Niazi | Similarity: 0.6641 | Wiki stakeholder: imran ahmad khan niazi
Closest Match:  dmytro lubinets -> Dmytro | Similarity: 0.5604 | Wiki stakeholder: dmytro
Closest Match:  lukpan ahmediyarov -> Ramzán Ajmádovich Kadírov | Similarity: 0.7442 | Wiki stakeholder: ramzan kadyrov
Closest Match:  marcin przydach -> Marcin Przybyłowicz | Similarity: 0.8186 | Wiki stakeholder: marcin przydacz
Closest Match:  joker dpr -> DPR | Similarity: 0.6407 | Wiki stakeholder: self-proclaimed donetsk people's republic
Closest Match:  khrystyna kit -> Kreml | Similarity: 0.4637 | Wiki stakeholder: th

In [21]:
for wiki_info in all_wiki_info:
    if ('wolodymyr') in wiki_info.lower():
        print(wiki_info)

Wolodymyr Hrojsman
Wolodymyr Boryssowytsch Hrojsman
Wolodymyr Hrojsman
Kateryna Wolodymyriwna Bondarenko
Kateryna Wolodymyriwna Wolodko
Wolodymyr Selenskyj
Wolodymyr Oleksandrowytsch Selenskyj
Witalij Wolodymyrowytsch Klytschko
Wolodymyr Selenskyj
Wolodymyr Oleksandrowytsch Selenskyj
Wolodymyr Selenskyj
Wolodymyr Oleksandrowytsch Selenskyj
Wolodymyr Selenskyj
Wolodymyr Oleksandrowytsch Selenskyj
Oleksandr Wolodymyrowytsch Schowkowskyj
Wolodymyr Selenskyj
Wolodymyr Oleksandrowytsch Selenskyj
Wolodymyr Selenskyj
Wolodymyr Oleksandrowytsch Selenskyj
Wolodymyr Lukaschenko
Wolodymyr Wolodymyrowytsch Lukaschenko
Wolodymyr Selenskyj
Wolodymyr Oleksandrowytsch Selenskyj
Serhij Wolodymyrowytsch Hajdaj
Wolodymyr Selenskyj
Wolodymyr Oleksandrowytsch Selenskyj
Witalij Wolodymyrowytsch Klytschko
Iryna Wolodymyriwna Dowhan
Wolodymyr Petrowitsch Schkidtschenko
Wolodymyr Rybak
Wolodymyr Iwanowytsch Rybak
Wolodymyr Selenskyj
Wolodymyr Oleksandrowytsch Selenskyj
Wolodymyr Selenskyj
Wolodymyr Oleksandrow

In [20]:
all_wiki_embeds_openai = get_embedding_openai(
    all_wiki_info,
    model="text-embedding-3-large",
    batch_size=1200
)

Embedding:   0%|          | 0/37 [00:00<?, ?it/s]

In [21]:
all_wiki_embeds_openai.shape

torch.Size([44351, 3072])

In [22]:
missing_stakeholder_embeddings_openai = get_embedding_openai(
    missing_stakeholders,
    model="text-embedding-3-large",
    batch_size=800
)

Embedding:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
missing_stakeholder_embeddings_openai.shape

torch.Size([1246, 3072])

In [24]:
search_results_openai = semantic_search(
    missing_stakeholder_embeddings_openai,  # query embeddings
    all_wiki_embeds_openai,  # database embeddings (can remain on CPU)
    top_k=1,  # Retrieve the top match only, adjust as needed,
    device='cuda',
)

Query chunks: 100%|██████████| 13/13 [00:01<00:00,  6.58it/s]


In [30]:
missing_stakeholder_matches_new = {}
missing_stakeholder_replacements_new = {}
for idx, result in enumerate(search_results_openai):
    best_match = result[0]
    similarity_score = best_match['score']
    best_match_idx = best_match['corpus_id']
    missing_stakeholder_matches_new[missing_stakeholders[idx]] = (all_wiki_info[best_match_idx], similarity_score, wiki_info_index_to_stakeholder[best_match_idx])
    print_string = "Mapping: " if similarity_score >= (1 - float(0.15)) else "Closest Match: "
    print(f"{print_string} {missing_stakeholders[idx]} -> {all_wiki_info[best_match_idx]} | Similarity: {similarity_score:.2f} | Wiki stakeholder: {wiki_info_index_to_stakeholder[best_match_idx]}")
    if similarity_score >= (1 - float(0.17)):
        stakeholder = missing_stakeholders[idx]
        best_match_stakeholder = wiki_info_index_to_stakeholder[best_match_idx]
        missing_stakeholder_replacements_new[stakeholder] = best_match_stakeholder

Closest Match:  abbas galyamov -> Aleksandr Gallyamov | Similarity: 0.60 | Wiki stakeholder: abbas galiamov
Closest Match:  dmytro finashyn -> Dmytro Yanchuk | Similarity: 0.70 | Wiki stakeholder: dmytro boyarchuk
Mapping:  aldeas infantiles sos -> Aldeas Infantiles SOS | Similarity: 0.90 | Wiki stakeholder: sos kinderdörfer
Closest Match:  zoya apostoliuk -> Olga Zaitseva | Similarity: 0.54 | Wiki stakeholder: olga zemtsova
Closest Match:  polish ambassador to canada -> Ministry of Foreign Affairs of Poland | Similarity: 0.65 | Wiki stakeholder: polish ministry of foreign affairs
Closest Match:  pierrette herzberger-fofana -> Danielle Bleitrach French sociologist | Similarity: 0.43 | Wiki stakeholder: danielle bleitrach
Closest Match:  ukrainian-born russian military officer -> Nikolai Ilyich Bulatov Russian military officer | Similarity: 0.61 | Wiki stakeholder: nicholas ivanovich bulaev
Closest Match:  sergej sobyanin -> Sergei Sobianin | Similarity: 0.83 | Wiki stakeholder: sergey 

In [31]:
openai_keys = set(missing_stakeholder_replacements_new.keys())
jina_keys = set(missing_stakeholder_replacements.keys())

print("In openai but not in jina: ", openai_keys - jina_keys)
for missing_key in openai_keys - jina_keys:
    print(f"{missing_key} -> {missing_stakeholder_replacements_new[missing_key]}")
print("____________"*20)
print("In jina but not in openai: ", jina_keys - openai_keys)
for missing_key in jina_keys - openai_keys:
    print(f"{missing_key} -> {missing_stakeholder_replacements[missing_key]} | {missing_stakeholder_matches[missing_key][1]}")

In openai but not in jina:  {'wolodimir zelenski', 'prigozhin supporters', 'kirilo budanov'}
wolodimir zelenski -> volodymyr zelenskiyy
prigozhin supporters -> yevgeny prigozhin
kirilo budanov -> kirill budanov
________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
In jina but not in openai:  {'pat ryder', 're: russia', 'elina beketova', 'artem dekhtiarenko', 'abbas galljamov', 'yurii ihnat', 'ukrainian-born russian military officer', 'german foreign minister annalena baerbock', 'herman haluschtschenko', 'oleksiy melnyk', "ukraine's partisans", 'ukrainian resistance', 'liudmyla huseinova', 'ukraine soldier', 'wagner private military company', 'anatoli antonow', 'andrii zalivskyi', 'pavlo lysianskyi', 'yeven olexienko', 'serhiy borsov', 'dmytro orlov', 'kassym-schomart tokajew', "ukraine's southern milit

In [32]:
missing_stakeholder_matches["wolodimir zelenski"]

("Volodimir Zelens'kij\u200f", 0.8285743594169617, 'volodymyr zelenskyi')

In [44]:
from collections import defaultdict

stakeholder_replacement_grouped = defaultdict(list)

for k, v in stakeholder_replacement.items():
    stakeholder_replacement_grouped[v].append(k)

In [52]:
for k, v in stakeholder_replacement_grouped.items():
    if len(v) > 1:
        print(f"{k} -> {v}")

Central Election Commission of Russia -> ['Central Election Commission of Russia', 'Central Election Commission of Ukraine']
Oleksandr Hranovskyi -> ['Oleksandr Hranovskyi', 'Andriy Hrinchenko', 'Serhiy Rybalka', 'Valeriy Fedorchuk', 'Dmytro Korkishko', 'Oleksiy Dytyatev', 'Stanislav Andreev', 'Oleksandr Stetsenko', 'Cezary Kucharski', 'Olesya Kurochkina', 'Vitaliy Kvashuk']
mykhailo podolyak -> ['mykhailo podolyak', 'mykhailo podoliak', 'mykhalio podlyak', 'mykahilo podolyak']
hanna malijar -> ['hanna malijar', 'hanna maliar', 'hanna maljar']
Embassy of Russia, Washington, D.C. -> ['Embassy of Russia, Washington, D.C.', 'Embassy of the United States, Moscow']
Aleksey Yevseyev -> ['Aleksey Yevseyev', 'Andrei Yegorychev', 'Oleksandr Yakovenko', 'Oleksandr Shovkovskyi', 'Ivan Yaremchuk']
kirilo budanov -> ['kirilo budanov', 'kyrylo budanov', 'kyrlo budanov', 'kryrylo budanov']
Oleksii Krasovskyi -> ['Oleksii Krasovskyi', 'Aleksey Kuznetsov', 'Oleksandra Kononova']
Oleksandr Korchmid -> [

In [49]:
key_we_are_looking_for = stakeholder_replacement["Volodymyr Zelenskyy"]
stakeholder_replacement_grouped[key_we_are_looking_for]

['Volodymyr Zelenskyy']